In [9]:
# ==========================================
# 1. Imports & Paths
# ==========================================

from pathlib import Path
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)

# --- Ensure python-chess is available (works in VS Code & Colab) ---
try:
    import chess
except ModuleNotFoundError:
    import sys, subprocess
    print("python-chess not found. Installing...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "python-chess", "--quiet"])
    except Exception:
        # fallback to user install if needed
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", "python-chess", "--quiet"])
    import chess
print("python-chess version OK.")

# --- Paths ---
NB_DIR   = Path.cwd()
DATA_DIR = (NB_DIR / "../data").resolve()
OUT_DIR  = DATA_DIR  # save cleaned files back to data/
OUT_DIR.mkdir(parents=True, exist_ok=True)

RAW_SUBSET = DATA_DIR / "chess_games_subset.csv"   # from 01_downsampling.ipynb
CLEAN_CSV  = DATA_DIR / "clean_chess_games.csv"
META_JSON  = DATA_DIR / "clean_metadata.json"

print("Data dir:", DATA_DIR)
print("Input:", RAW_SUBSET)

python-chess version OK.
Data dir: E:\Github Projects\chess-outcome-prediction\data
Input: E:\Github Projects\chess-outcome-prediction\data\chess_games_subset.csv


In [8]:
# ==========================================
# 2. Load Subset, Define Target, Verify Moves Column
# ==========================================

assert RAW_SUBSET.exists(), f"{RAW_SUBSET} not found. Run 01_down_sampling.ipynb first."

# Load subset CSV
df_raw = pd.read_csv(RAW_SUBSET)
print("Raw shape:", df_raw.shape)
display(df_raw.head(3))

# ---- Target mapping ----
target = None
for cand in ["target", "Result", "result"]:
    if cand in df_raw.columns:
        if cand.lower() == "result":
            target = df_raw[cand].map({"1-0": "white", "0-1": "black", "1/2-1/2": "draw"})
        else:
            target = df_raw[cand]
        break
assert target is not None, "Could not locate a result/target column in subset."

df_raw["target"] = target.astype(str)
print("Target distribution:")
display(df_raw["target"].value_counts())

# ---- Find moves column & quick stats ----
moves_col = None
for cand in ["Moves", "moves", "PGN", "pgn", "AN"]:
    if cand in df_raw.columns:
        moves_col = cand
        break
assert moves_col is not None, "Could not find a moves column (e.g., 'Moves')."

print(f"Using moves column: '{moves_col}'")

# Rough ply-count estimate from SAN: count tokens that are not move numbers like '1.' '2.' etc.
def approx_plies(s):
    if not isinstance(s, str):
        return 0
    toks = [t for t in s.replace("\n", " ").split(" ") if t and not t[0].isdigit()]
    return len(toks)

ply_counts = df_raw[moves_col].fillna("").map(approx_plies)
total_games = len(ply_counts)
short_games = int((ply_counts < 30).sum())  # < 30 plies == < 15 moves total
print(f"Games with < 15 moves (kept): {short_games} / {total_games} ({short_games/total_games:.1%})")
print(f"Median plies: {int(ply_counts.median())}, Mean plies: {ply_counts.mean():.1f}")

Raw shape: (119868, 15)


,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Bullet,MM2010,behrang71,1-0,2016.07.01,10:30:21,1812,1803,10.0,-10.0,A03,Bird Opening: Dutch Variation,120+0,Normal,1. f4 d5 2. g3 c5 3. Bg2 Nc6 4. Nf3 Bf5 5. O-O...
1,Blitz,tgnoo,janbubba,1-0,2016.07.01,11:01:23,1840,1791,10.0,-9.0,B00,Owen Defense,180+0,Normal,1. e4 b6 2. d4 Bb7 3. Bd3 e6 4. f4 d6 5. Nf3 h...
2,Bullet,Chaliko,TheGameHen,0-1,2016.07.01,00:27:23,2193,1782,-25.0,20.0,E20,Nimzo-Indian Defense #2,60+2,Time forfeit,1. d4 Nf6 2. c4 e6 3. Nc3 c5 4. d5 d6 5. e3 e5...


Target distribution:


target
white    59875
black    55329
draw      4640
nan         24
Name: count, dtype: int64

Using moves column: 'AN'
Games with < 15 moves (kept): 10672 / 119868 (8.9%)
Median plies: 70, Mean plies: 94.8


In [10]:
# ==========================================
# 3. First-15-Moves Feature Extraction (ply = 30)
# ==========================================

# If the raw dataset uses 'AN' for the moves, rename it now for consistency
if "AN" in df_raw.columns:
    df_raw = df_raw.rename(columns={"AN": "Moves"})
    print("Renamed 'AN' -> 'Moves'")

# Confirm moves column exists
assert "Moves" in df_raw.columns, "Expected a 'Moves' column after renaming from 'AN'."

import chess

def features_from_ply30(moves_san: str) -> dict:
    """
    Build a compact, leakage-safe feature vector from the position after ply 30
    (i.e., 15 full moves), or earlier if the game is shorter.
    """
    # Guard against missing/NaN
    if not isinstance(moves_san, str) or not moves_san.strip():
        moves_san = ""

    board = chess.Board()
    captures = 0
    checks = 0

    # Tokenise SAN and filter out move numbers like '1.' '2.' etc.
    tokens = [t for t in moves_san.replace("\n", " ").split(" ") if t and not t[0].isdigit()]
    ply_limit = min(30, len(tokens))  # 30 ply == 15 moves total

    for i in range(ply_limit):
        san = tokens[i]
        try:
            move = board.parse_san(san)
        except Exception:
            # If a token can’t be parsed, stop at the last valid position
            break
        if board.is_capture(move):
            captures += 1
        board.push(move)
        if board.is_check():
            checks += 1

    # --- Feature block from board state at ply_limit ---
    def count(piece, colour):
        return len(board.pieces(piece, colour))

    feat = {}
    # Piece counts
    for piece, name in [
        (chess.PAWN, "P"), (chess.KNIGHT, "N"), (chess.BISHOP, "B"),
        (chess.ROOK, "R"), (chess.QUEEN, "Q")
    ]:
        feat[f"w_{name}"] = count(piece, chess.WHITE)
        feat[f"b_{name}"] = count(piece, chess.BLACK)

    # Material (simple values) & balance
    val = {"P": 1, "N": 3, "B": 3, "R": 5, "Q": 9}
    w_mat = sum(feat[f"w_{k}"] * v for k, v in val.items())
    b_mat = sum(feat[f"b_{k}"] * v for k, v in val.items())
    feat["material_white"]   = w_mat
    feat["material_black"]   = b_mat
    feat["material_balance"] = w_mat - b_mat

    # Side to move & castling rights
    feat["stm_white"] = int(board.turn == chess.WHITE)
    feat["wk_castle"] = int(board.has_kingside_castling_rights(chess.WHITE))
    feat["wq_castle"] = int(board.has_queenside_castling_rights(chess.WHITE))
    feat["bk_castle"] = int(board.has_kingside_castling_rights(chess.BLACK))
    feat["bq_castle"] = int(board.has_queenside_castling_rights(chess.BLACK))

    # Centre control & mobility proxies
    centres = [chess.D4, chess.E4, chess.D5, chess.E5]
    feat["w_centre_ctrl"] = sum(board.is_attacked_by(chess.WHITE, sq) for sq in centres)
    feat["b_centre_ctrl"] = sum(board.is_attacked_by(chess.BLACK, sq) for sq in centres)

    # Mobility measured at current position
    # (Count legal moves for side to move, and approximate for the other by toggling turn)
    if board.turn == chess.WHITE:
        feat["w_mobility"] = board.legal_moves.count()
        board.turn = chess.BLACK
        feat["b_mobility"] = board.legal_moves.count()
        board.turn = chess.WHITE
    else:
        feat["b_mobility"] = board.legal_moves.count()
        board.turn = chess.WHITE
        feat["w_mobility"] = board.legal_moves.count()
        board.turn = chess.BLACK

    # Cumulative dynamics up to ply 30
    feat["captures_upto30"] = captures
    feat["checks_upto30"]   = checks
    feat["ply_processed"]   = ply_limit
    return feat

# Apply feature extraction
feats = df_raw["Moves"].fillna("").apply(features_from_ply30)
feat_df = pd.DataFrame(list(feats))

print("First-15-moves feature shape:", feat_df.shape)
display(feat_df.head(3))


Renamed 'AN' -> 'Moves'
First-15-moves feature shape: (119868, 25)


,w_P,b_P,w_N,b_N,w_B,b_B,w_R,b_R,w_Q,b_Q,material_white,material_black,material_balance,stm_white,wk_castle,wq_castle,bk_castle,bq_castle,w_centre_ctrl,b_centre_ctrl,w_mobility,b_mobility,captures_upto30,checks_upto30,ply_processed
0,5,6,2,1,2,2,2,2,1,1,36,34,2,1,0,0,1,0,4,4,48,46,6,0,30
1,6,7,2,1,1,2,2,2,1,1,34,35,-1,1,0,0,0,0,1,3,46,29,5,2,28
2,8,7,1,1,1,2,2,2,1,1,33,35,-2,1,0,0,0,0,4,3,37,31,4,0,30
